<a href="https://colab.research.google.com/github/tqmbow/Sentinel1Clearfelling/blob/master/S1ClassifiersSciKit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split

In [0]:
! pip install geopandas
! pip install descartes
! pip install rasterio

In [0]:
import geopandas as gpd
import descartes
import rasterio
from rasterio.mask import mask
from rasterio.features import geometry_mask
from shapely.geometry import mapping

In [0]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


##Set Study Region

In [0]:
region = 'ScotlandRSRegion4'
wd= '/content/drive/My Drive/colab_data/{}'.format(region)

Buffer polygons by -10 to remove edge pixels; split into training and testing

In [0]:
#extent = gpd.read_file(os.path.join(wd, '{}_extent.shp'.format(region)))
felled = gpd.read_file(os.path.join(wd, '{}_felled.shp'.format(region)))
mature = gpd.read_file(os.path.join(wd, '{}_mature.shp'.format(region)))

print("mature:", mature["geometry"].area.sum()/10000, "ha")
print("felled:", felled["geometry"].area.sum()/10000, "ha")

#Buffer -10 and save output
mature["geometry"] = mature.geometry.buffer(-10)
felled["geometry"] = felled.geometry.buffer(-10)
mature = mature.explode()
felled = felled.explode()

print("mature buffered:", mature["geometry"].area.sum()/10000, "ha")
print("felled buffered:", felled["geometry"].area.sum()/10000, "ha")

mature = mature[mature["geometry"].area >=1000]
felled = felled[felled["geometry"].area >= 1000]

trainFelled, testFelled = train_test_split(felled, test_size=0.2, random_state = 999)
trainMature, testMature = train_test_split(mature, test_size = 0.2, random_state =999)
#matureBuffer = [f for f in matureBuffer.geometry.values if f.area > 1000]
#felledBuffer = [f for f in felledBuffer.geometry.values if f.area > 1000]
print(trainFelled.shape)
print(testFelled.shape)


mature: 9770.950109324482 ha
felled: 2601.968319982719 ha
mature buffered: 8165.301212508738 ha
felled buffered: 2129.4909402684143 ha
(284, 4)
(72, 4)


Read the S1 Image

Extract pixels within polygons to build training and testing features

In [0]:
#Read image
s1 = rasterio.open(os.path.join(wd,'rsRegion4_3month16BALLs.tif'))

#Print number of bands
B = s1.count
print(B)
s1prof = s1.profile.copy()

12


In [0]:
s1.read()

array([[[        nan,  -6.7598996,  -8.562072 , ...,  -8.2013235,
          -9.046654 , -10.582595 ],
        [ -7.6428266,  -9.736197 , -10.241104 , ...,  -8.867673 ,
         -10.377863 , -11.258421 ],
        [ -9.816462 ,  -9.809568 , -10.371953 , ...,  -9.205761 ,
         -10.807577 , -11.7656555],
        ...,
        [-17.474506 , -17.014412 , -15.526152 , ..., -12.128495 ,
         -10.86965  , -11.094606 ],
        [-17.284782 , -17.075914 , -15.361743 , ..., -11.948948 ,
         -10.784105 , -11.632518 ],
        [-16.695944 , -17.253906 , -16.103554 , ..., -11.452044 ,
         -11.53695  , -12.456648 ]],

       [[        nan, -15.8581705, -16.916975 , ..., -18.37664  ,
         -18.903446 , -18.596397 ],
        [-16.936644 , -17.98128  , -19.104717 , ..., -17.154308 ,
         -18.981857 , -20.271921 ],
        [-17.03609  , -17.795042 , -17.771828 , ..., -16.973566 ,
         -17.307741 , -18.524313 ],
        ...,
        [-22.545631 , -22.56948  , -21.306974 , ..., -

In [0]:
def getPixels(image, poly, target):
    global B
    shape=[mapping(poly)]
    outImage, out_transform = mask(image, shape, crop=True, nodata=999) 
    outList = outImage.reshape((B, -1)).T
    return np.append(target, outList, axis=0)


def extractAllPolys(image, features):
    global B
    flatten = np.array([]).reshape(0,B)
    for f in features:
        if f.area > 0:
            flatten = getPixels(image,f, flatten)
    flattenArr = np.ma.masked_array(flatten, mask=(flatten == 999))
    return pd.DataFrame(flattenArr).dropna()
      

In [0]:
trainFelledF = extractAllPolys(s1,trainFelled.geometry.values)
testFelledF = extractAllPolys(s1,testFelled.geometry.values)
print("train felled size", trainFelledF.shape)
print("test felled size", testFelledF.shape)
trainMatureF = extractAllPolys(s1,trainMature.geometry.values)
testMatureF = extractAllPolys(s1,testMature.geometry.values)
print("train mature size", trainMatureF.shape)
print("test mature size", testMatureF.shape)

train felled size (175084, 12)
test felled size (37544, 12)
train mature size (682657, 12)
test mature size (131994, 12)


In [0]:
trainMatureF = trainMatureF.sample(trainFelledF.shape[0])
testMatureF = testMatureF.sample(testFelledF.shape[0])
print(trainMatureF.shape)
print(testMatureF.shape)

(175084, 12)
(37544, 12)


In [0]:
trainMatureF["TYPE_CODE"] = 0
testMatureF["TYPE_CODE"] = 0
trainFelledF["TYPE_CODE"] = 1
testFelledF["TYPE_CODE"] = 1

In [0]:
train = pd.concat([trainMatureF, trainFelledF])
test = pd.concat([testMatureF, testFelledF])

Drop the labels from trainX, vector as trainy

In [0]:
trainX = train.drop("TYPE_CODE", axis=1)
trainy = train["TYPE_CODE"].copy()

testX = test.drop("TYPE_CODE", axis = 1)
testy = test["TYPE_CODE"].copy()
trainX.head()

,0,1,2,3,4,5,6,7,8,9,10,11
107684,-12.623568,-16.506538,-7.986333,-12.791503,-15.158631,-17.788149,-11.580701,-14.709453,-6.442196,-10.756843,-10.043466,-15.415751
39490,-11.869100,-15.138095,-10.402539,-14.027419,-14.686519,-20.519230,-11.758499,-15.901530,-9.189785,-12.334918,-9.452192,-13.386502
1315625,-7.501197,-12.268439,-13.331161,-18.217167,-9.167229,-15.073679,-18.463055,-19.918652,-12.457465,-16.182779,-6.290135,-10.709098
2042749,-8.832651,-14.123918,-8.330820,-12.542852,-13.802910,-17.174698,-12.171342,-15.053130,-6.904105,-10.974566,-6.733414,-10.126642
1965243,-13.525480,-16.382107,-11.822089,-15.134338,-15.288528,-19.450205,-13.360148,-19.576237,-8.310637,-13.443035,-11.468226,-14.763532


Scale the pixel values using standard scaler

In [0]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
trainX = sc.fit_transform(trainX)
testX = sc.transform(testX)

In [0]:
trainX[10000]

array([-0.92370901, -0.94297347,  0.68282292,  0.85493831, -0.69318545,
       -0.29614022,  0.63581854,  1.65680049,  0.31973843,  0.20941968,
       -1.19729799, -1.3915716 ])

In [0]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 0.95)
print(trainX[0])
X_reduced = pca.fit_transform(trainX)
X_reduced[0]
pca.components_

[-1.26500627 -0.60472627  0.76034599  1.25285703 -1.16524     0.10355055
  0.24411584  1.51149797  0.36897553  1.02250587 -1.11493177 -1.20633512]


array([[-0.31729562, -0.29873068,  0.29639908,  0.2727176 , -0.30061519,
        -0.27057017,  0.28544989,  0.25099572,  0.28742248,  0.27039135,
        -0.31127401, -0.29526818],
       [-0.26916083, -0.30089764, -0.29039459, -0.3266415 , -0.26696479,
        -0.28299866, -0.28495523, -0.29871142, -0.27595637, -0.3168983 ,
        -0.25755695, -0.28511558],
       [ 0.23823055, -0.183894  ,  0.33781409, -0.30292309,  0.19841923,
        -0.31707232,  0.27949968, -0.47179872,  0.39244106, -0.17445748,
         0.25987017, -0.10464887],
       [ 0.10583079,  0.07629576, -0.10403049,  0.13535671, -0.29730721,
        -0.66671642, -0.2520776 , -0.03058927, -0.03565892,  0.31392477,
         0.30621144,  0.40356916],
       [-0.2857122 ,  0.33172902,  0.15181026, -0.1600368 , -0.50183644,
         0.22951319,  0.21522501, -0.33957551,  0.16160832, -0.10278042,
        -0.19703968,  0.4669166 ],
       [ 0.05051616,  0.05426704,  0.03240995, -0.10038627, -0.13320793,
        -0.25198664,  

Random Forest

In [0]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators = 10, random_state = 99)
rf.fit(trainX, trainy)
pred_y = rf.predict(testX)
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(testy, pred_y)
acc = accuracy_score(testy, pred_y)
print ("RF:\n{0}\n{1}%".format(cm,round(acc*100, 3)))

RF:
[[31946  5598]
 [11169 26375]]
77.67%


KNN

In [0]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 2, metric = 'minkowski', p = 2)
knn.fit(trainX, trainy)
pred_y = knn.predict(testX)
cm = confusion_matrix(testy, pred_y)
acc = accuracy_score(testy, pred_y)
print ("KNN:\n{0}\n{1}%".format(cm,round(acc*100, 3)))

Logistic regression

In [0]:
from sklearn.linear_model import LogisticRegression
lgr = LogisticRegression(random_state = 0)
lgr.fit(trainX, trainy)
pred_y = lgr.predict(testX)
cm = confusion_matrix(testy, pred_y)
acc = accuracy_score(testy, pred_y)
print ("logistic regression:\n{0}\n{1}%".format(cm,round(acc*100, 3)))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


logistic regression:
[[32090  5454]
 [ 9929 27615]]
79.513%


In [0]:
pred_y = lgr.predict_proba(testX).astype('float32')
pred_y[1]

XGBoost

In [0]:
from xgboost import XGBClassifier
xg = XGBClassifier()
xg.fit(trainX, trainy)
pred_y = xg.predict(testX)
cm = confusion_matrix(testy, pred_y)
acc = accuracy_score(testy, pred_y)
print ("XGBoost:\n{0}\n{1}%".format(cm,round(acc*100, 3)))

Ensemble classifier

In [0]:
from sklearn.ensemble import VotingClassifier
voting_clf = VotingClassifier(estimators=[('lr', lgr),('rf', rf),('knn',knn),('xg',xg)],voting='hard')
voting_clf.fit(trainX, trainy)
pred_y = voting_clf.predict(testX)
cm = confusion_matrix(testy, pred_y)
acc = accuracy_score(testy, pred_y)
print ("Ensemble:\n{0}\n{1}%".format(cm,round(acc*100, 3)))

In [0]:
breAK ME

##  Fit Classifiers to Image

In [0]:
#Classified Output
s1prof.update(count=1, nodata=None, dtype=np.int32)
dst = rasterio.open(os.path.join(wd,'{}3month_outputClassLgr.tif'.format(region)), 'w', **s1prof)

model = lgr #xg, rf, knn, lgr

for block_index, window in s1.block_windows(1):
    s1_block = s1.read(window=window, masked=True)
    v = s1_block.shape
    s1_block = np.nan_to_num(s1_block)
    s1_block = s1_block.reshape(v[0], -1).T
    s1_block = sc.transform(s1_block)
    result_block = model.predict(s1_block).reshape(1,v[1],v[2]).astype(np.int32)
    dst.write(result_block, window=window)
dst.close()

In [0]:
r = s1.read()

In [0]:
#Probability Output
s1prof.update(count=1, nodata=None, dtype=np.float32)
dst = rasterio.open(os.path.join(wd,'{}3month_outputProbLgr2.tif'.format(region)), 'w', **s1prof)

model = lgr #xg, rf, knn, lgr

for block_index, window in s1.block_windows(1):
    s1_block = s1.read(window=window, masked=True)
    v= s1_block.shape
    s1_block = s1_block.reshape(v[0], -1).T
    s1_block = np.nan_to_num(s1_block)
    s1_block = sc.transform(s1_block)
    result_block = model.predict_proba(s1_block).astype('float32')
    result_block = result_block[:,1:]
    result_block = result_block.reshape(1,v[1],v[2])
    dst.write(result_block, window=window)
dst.close()


In [0]:
from rasterio.plot import show

prob = rasterio.open(os.path.join(wd,'{}3month_outputProb.tif'.format(region))).read()

show(prob)

#prob.shape

In [0]:
import skimage.segmentation as seg

watershed = seg.watershed(prob, markers=, connectivity=1, offset=None, mask=None, compactness=5, watershed_line=False)



In [0]:
watershed.shape
watershed[:,1,1]